#  ספר 2: ספר על קוד ריצה במקביל 

ספר זה אנו נסביר את האלגורתמיקה של התיכנות במקביל שלנו ומה כל פונקציה עושה. ניתן להריץ את הפונקציות אחד אחד בשביל לראות את התוצאות לשהם וזאת הסיבה למה השתמשנו ב-jupyter notebook

##  Import:
בחלק זה אנחנו טוענים את כל הסיפריות והמשתנים הגלובים השתמשנו כדי ליצור את המערכת ריצת קוד במקביל

In [2]:
import os #סיפריה אשר נותנת לנו פעלות של מערכת הפעלה
import multiprocessing# ספרייה שנותמת לנו ליצור פרוסס חדש במערכת הפעלה
import time #ספרייה שנותנת לנו אפשרות לבצע פעולות על זמן
import numpy as np #ספרייה הרחבה לפעולות פעולות מתמטיקאיות על מערכים וכ'ו
import threading #ספרייה אשר נותנת לנו לעשות פעולות אם תהליכונים
import pickle #ספרייה עזר ליצרית קבצים
import Pdf2Text # פעלות של ספר מספר 1
import heapq #סיפרייה של פעולות על תור עדיפויות
import config # דף הגדרות של הפריוקט
theadcount = 0 #משתנה גלובלי אשר סופר כמה תהליכונים פעלים
processBagofwords = {} # משתנה גלובלי אשר מחזיק את רשימת המילים הכי שחיכות בתהליך מסויים
prossesstokenarry =[] # משתנה גלובלי אשר מחזיק את המידע התקבל אחרי הפעולת יצרית ווקטור מילים שקורא בתהליך מסויים

## תהליך: יצרית תהליכונים קטנים
הפעולה מקבלת מערך של שלמות קבצים,כתובת התקייה שהקבצים נמצאים בא,מליון תהליכים,ומפתח תהליכים
 
 procnum, return_dict, root,                  files
 
 הפעולה מחקלת את המערך הקבצים לעשרה מערכים שונים. ויוצרת תהליכונים.
 
 הפעולה מחזירה לתוך מילון התהליך את רשימה שהתקבלה אחרי תהליך המתבצי בספר 1.ווקטור מילים המסודרים לפי המילה הכי שכיחה להכי נדירה.

In [3]:
def process(files,root,return_dict,procnum):
    if(len(files)>0):
        Bagofwords={}
        theadsarry =[]
        tokenarry = []
        global processBagofwords
        global prossesstokenarry
        processlock = threading.Lock()
        global theadcount
        splitedfilses = np.array_split(files,10)
        for newfiles in splitedfilses: #לולאה היוצרת תהליכון לכל מערך
            t1 = threading.Thread(target=Theard, args=(root,newfiles,processlock,))
            theadsarry.append(t1)
            theadcount+=1
        time.sleep(0.5)
        for t in theadsarry: #לולאה אשר מפעילה את המערך
            t.start()
        while(theadcount>0): #לולאה אשר בודקת אם כל התהליכונים עדיין בחיים ועובדים
            if( len(processBagofwords)>0): #בדיקה עם יש מילים בתוך ווקטור המילים השכיחות
                processlock.acquire()
                Bagofwords = Pdf2Text.margedictionaries(Bagofwords,processBagofwords)
                processBagofwords= {}
                processlock.release()
            if(len(prossesstokenarry)>0): #בדיקה אם יש את הרישמה הנוצרת בספר 1
                processlock.acquire()
                tokenarry += prossesstokenarry
                prossesstokenarry=[]
                processlock.release()
        return_dict[procnum] = [tokenarry,Bagofwords]

## תהליכון: המרת/נקוי המילים מהקבוץ pdf ויצרת ווקטור מילים
הפעולה מקבלת מערך של שלמות קבצים,כתובת התקייה שהקבצים נמצאים בא, מפתח לסמפור

root,files,processlock

פעולה מפעילה את פעולת המרת המילים מספר 1 תוך כדי סיכרון עם כל התהליכונים האחרים

הפעולה מחזירה את הרשימה שהתקבלה בספר 1 ואת הרשימת המילים השכיחה ביותר בתהליכון המסויים זה.

In [4]:
def Theard(root,files,processlock,):
    global processBagofwords
    global prossesstokenarry
    global theadcount
    theardbagofword = {}
    tokenarry = []
    try:
        for name in files:  #לולאה שרצה על כל המסמכים ויוצרת האת המודל של ספר 1
            bagofword = Pdf2Text.BuildModelToTraing(os.path.join(os.path.join(root, name)), name, processlock, ) #פעולה מספר 2
            theardbagofword = Pdf2Text.margedictionaries(theardbagofword, bagofword[1])
            tokenarry.append(bagofword[0])
        processlock.acquire() #סמפור
        processBagofwords = Pdf2Text.margedictionaries(processBagofwords, theardbagofword)
        prossesstokenarry += tokenarry
        theadcount -= 1
        processlock.release()
    except Exception as err:
        processlock.acquire()
        theadcount -= 1
        processlock.release()
        raise err


## יצירת תהליכים:
 הפעולה מקבלת את מספר הליבות במעמד, מיקום התקיות, רפרנס למספר הפרוסס, מילון החזרה של תהליך, ומערך של תהליכים פעילים
 
 פעולה הזאת זאת יוצרת את התהליכונים החדשים לפי מספר הליבות במערכת בשביל לא ליצור עומס

In [5]:
def Startingproses(cpu_count, dircount, prossesid, return_dict, theardonjobnow):
    for baseroot, directories, nonfiles in os.walk(os.path.join(os.path.join(config.WORKSPACE,config.TRANING)), topdown=False):
        #לולא שבודקת את התקיות שהוגדר באזור העבודה
        allfiles = []
        for root, dirs, files in os.walk(baseroot):#הלולא עוברת על כל קובץ בתקיות
            for f in files:# הלולאה מוספיה כל קובץ לתוך מערך קבצים של כל תקייה
                allfiles.append(f)
            splitefile = np.array_split(files, cpu_count / dircount)# פעולה מפצלת את המערך למערכים קטנים לפי כמות הליבות
            for filestosend in splitefile: #הלולאה יוצרת תהליך חדש
                if (baseroot == (os.path.join(config.WORKSPACE,config.TRANING))):
                    p1 = multiprocessing.Process(target=process, args=(filestosend, root, return_dict, prossesid))
                    theardonjobnow.append(p1)
                    prossesid = prossesid + 1
    for p in theardonjobnow:#הפעולה מפעילה את כל התהליכים שנוצרו
        p.start()


## ניהול התהליכים
פעולה זאת מקבלת מערך ריק של המילים הכי שחיכות,רשימה ריקה של התהליך שקורה בספר 1,מספר הליבות במעמד, מיקום התקיות, רפרנס למספר הפרוסס, מילון החזרה של תהליך, ומערך של תהליכים פעילים

הפעולה יוצרת ונהלת את על התהליכים עד שהם מסתיימים.ושומרת בקובץ את המערך המילים השיכוחת ואת מערך הרשימות של ספר 1

In [10]:
def BuildBagOfWord(Bagofwords, cpu_count, dircount, i, return_dict, theardonjobnow, tokenarry):
    for baseroot, directories, nonfiles in os.walk(os.path.join(os.path.join(config.WORKSPACE,config.TRANING)), topdown=False):
        #לולאה בודק איזה סוג מסמך זה על פי שם תקייב
        if len(directories) > dircount:
            dircount = len(directories)
    Startingproses(cpu_count, dircount, i, return_dict, theardonjobnow) #הפעולה יוצרת תהליכים
    while (len(theardonjobnow) > 0): #הלולאה הזאת מבצבם שח כל התהליכים עד שהם מסתיימים
        for poress in theardonjobnow:# הלולאה בודקת אם כל התהלכים בחיים אחרת מוציאה אותו מהמערכך
            if poress.is_alive() == False:
                theardonjobnow.remove(poress)
    time.sleep(0.2)
    for x in return_dict: # הלולאה לוחקת את מה שהוחזר המפרוסס ומאחדת אותם למערכים שלמים אחד של במילים ואחד של הרשימות
        Bagofwords = Pdf2Text.margedictionaries(return_dict[x][1], Bagofwords)
        tokenarry += return_dict[x][0]
    most_freq = heapq.nlargest(900, Bagofwords, key=Bagofwords.get) #יציאת ערמה של המלים רק ל900 המלילים הכי שחיכות מהמליון שחיכיות
    file = open(os.path.join(config.WORKSPACE,config.BAGOFWORDSFILE), 'wb')
    pickle.dump(most_freq, file)
    file = open(os.path.join(config.WORKSPACE,config.TOKENTOTRAINFILE), 'wb')
    pickle.dump(tokenarry, file)
    #יצירת שני קבצים של ערמת שחיכיות שאחר כך תהפוך template
    #יצרית הרשימה של ספר 1 בשביל להמשיך התהליך של לימוד מוכנה

## הוספת input למכונה
הפעולה הזאת מקבלת את הרשימה מספר 1 ומוסיפה קלט למכונה 

In [7]:
def GetDataFromFile(tokentotrain):
    try:
        traningset = []
        file = open(os.path.join(config.WORKSPACE, config.BAGOFWORDSFILE), 'rb')
        Bagofwords = pickle.load(file)
        file.close()
        if (tokentotrain == None):
            file = open(os.path.join(config.WORKSPACE, config.TOKENTOTRAINFILE), 'rb')
            tokentotrain = pickle.load(file)
            file.close()
            #פתחת את הקבצים שיצרנו בפעולה הקודמת
        for file in tokentotrain: #הלולאה רוצה על כל הtemplate
            data = file
            sent_vec = []
            for word in Bagofwords:
                if word in file[2]: #התנאי בודק את המילים שבטקסט ובודק עם הנמצאים מסנן אותם באחד במקום של המילה אחרת נסמן ב0
                    sent_vec.append(1)
                else:
                    sent_vec.append(0)
            sent_vec = np.asarray(sent_vec)
            data.append(sent_vec)
            traningset.append(data)
        return (Bagofwords, traningset)
    except:
        return config.ERRORLABEL


## פונקציה לחישוב מקבלי
הפונקציה הזאת מחברת את כל הפנקוציות שיצרנו ויצורת מהם מאגר של מילים שחיכות וגם יוצרת רשימה של ספר 1 עם אינפוט למכונה.

שניי הפנקציות עושות אותו הדבר אבל אחת יש פרמטר שאומר את המיקום ועושה את הפעולה הזאת אך ללא שמומש בתהלכים אלא רק בתהליכונים

In [8]:
def begofword():
    cpu_count = multiprocessing.cpu_count()
    Bagofwords = {}
    tokenarry = []
    theardonjobnow = []
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    dircount = 0
    i = 0
    if(not os.path.exists(os.path.join(config.WORKSPACE, config.BAGOFWORDSFILE))):
        BuildBagOfWord(Bagofwords, cpu_count, dircount, i, return_dict, theardonjobnow, tokenarry)
    return GetDataFromFile(None)


def bagofwordonlytheard(when):
    Bagofwords = {}
    theadsarry = []
    tokenarry = []
    global processBagofwords
    global prossesstokenarry
    global theadcount
    processlock = threading.Lock()
    allfiles = []
    for root, directories, files in os.walk(when, topdown=False):
        allfiles += (files)
    if len(allfiles) >2:
        splitedfilses = np.array_split(allfiles, len(allfiles) / 2)
        for newfiles in splitedfilses:
            t1 = threading.Thread(target=Theard, args=(root, newfiles, processlock,))
            theadsarry.append(t1)
            theadcount += 1
    else:
        splitedfilses =allfiles
        t1 = threading.Thread(target=Theard, args=(root, splitedfilses, processlock,))
        theadsarry.append(t1)
        theadcount += 1
    time.sleep(0.5)
    for t in theadsarry:
        t.start()
    while (theadcount > 0):
        if (len(processBagofwords) > 0):
            processlock.acquire()
            Bagofwords = Pdf2Text.margedictionaries(Bagofwords, processBagofwords)
            processBagofwords = {}
            processlock.release()
        if (len(prossesstokenarry) > 0):
            processlock.acquire()
            tokenarry += prossesstokenarry
            prossesstokenarry = []
            processlock.release()

    return GetDataFromFile(tokenarry)[1]

אם נילחץ על הקוד כאן למטה יהיה אפשר לראות את מה מחזירה הפונקציה

לפעולה הזאת לוקחת החן 1-5 דקות

In [9]:
import warnings
warnings.filterwarnings("ignore")
print(bagofwordonlytheard(os.path.join("exmple")))

[['לבדיקה', 'exmple\\2015.pdf', ['טופס', 'אל', 'שלוש', 'מאות', 'ואחד', 'שודר', 'עי', 'מייצג', 'באינטראנט', 'אחד', 'מתו', 'ארבעה', 'התשמיוס', 'ליחיד', 'וו', 'וו', 'אסמכתא', 'חותמת', 'המשרד', 'הרשו', 'בלשו', 'זכר', 'מתיחס', 'לנקבה', 'די', 'וחשבו', 'על', 'ההכנסות', 'באר', 'ובחול', 'תשע', 'תשע', 'תארי', 'הגשת', 'הדוח', 'ור', 'הדוח', 'בשנת', 'המס', 'אלפיי', 'ושמונהעשרה', 'שתיי', 'השנה', 'המתחילה', 'והמסתיימת', 'השומה', 'תל', 'אביב', 'חמש', 'חוליה', 'שבעה', 'עפי', 'סעי', 'מאה', 'ושלושי', 'ואחת', 'לפקודת', 'מס', 'הכנסה', 'רשאי', 'פקיד', 'השומה', 'לראות', 'מי', 'שהגיש', 'דוח', 'שלא', 'כלליי', 'מולא', 'כראוי', 'או', 'שלא', 'צורפו', 'אליו', 'המסמכי', 'המתאימי', 'כמי', 'שלא', 'הגיש', 'דוח', 'על', 'הכנסותי', 'והכנסות', 'בת', 'זוגי', 'שלושה', 'הכנסותי', 'בלבד', 'חמש', 'אני', 'מגיש', 'דוח', 'לשנת', 'מס', 'זו', 'למרות', 'שאיני', 'חייבבקשה', 'להחוזר', 'מס', 'וגי', 'מגיש', 'דוח', 'נפרד', 'מצב', 'הדוחהצהרה', 'של', 'בת', 'זוגי', 'אי', 'הכנסה', 'לב', 'בת', 'זוגי', 'בת', 'זוגי', 'עזר', 'לי', 'בהשגת', 'הכנס

In [8]:
import os
import config
import pickle
file = open(os.path.join(config.WORKSPACE, config.BAGOFWORDSFILE), 'rb')
Bagofwords = pickle.load(file)
file.close()
print(Bagofwords)

['על', 'טופס', 'בת', 'מספר', 'סעי', 'מצב', 'אל', 'מאות', 'לי', 'לפקודה', 'המס', 'זוגי', 'אחד', 'של', 'או', 'עשרה', 'הזוג', 'בשנת', 'מס', 'תארי', 'רכוש', 'זה', 'מאה', 'הנני', 'חמש', 'הכנסה', 'לפי', 'מבוסס', 'לא', 'דוח', 'שלושה', 'כתובת', 'מאתיי', 'עשרי', 'את', 'שמונה', 'עלי', 'היו', 'שתיי', 'שלוש', 'וחמש', 'שבע', 'הדוח', 'הכנסות', 'הרשו', 'ארבע', 'תושב', 'תיק', 'סהייכ', 'ארבעי', 'לפקודת', 'כולל', 'לב', 'בסעי', 'וחשבו', 'אפס', 'שש', 'שח', 'עוסק', 'מאז', 'חשבו', 'קבוע', 'פרטי', 'וארבע', 'בהתא', 'ואחת', 'מיליו', 'בנאמנות', 'ושש', 'הכנסותי', 'עפי', 'חייבת', 'תשע', 'לשותפות', 'נער', 'עד', 'ספירה', 'הערכה', 'שוט', 'הפרטי', 'בעל', 'בעסק', 'קופה', 'ושמונה', 'יש', 'די', 'בשותפות', 'אני', 'שלא', 'בלבד', 'חשבונות', 'במעמ', 'יותר', 'אלפיי', 'לצר', 'ושלושי', 'נהנה', 'כי', 'המסמכי', 'המסי', 'בדיווח', 'מי', 'שאינו', 'חתימת', 'ושלוש', 'רשות', 'בדוח', 'בחול', 'מתחדשות', 'החשבו', 'המשרד', 'דואר', 'חוק', 'הו', 'ושניי', 'ניכויי', 'ותשע', 'חמישי', 'אחר', 'מלאי', 'לכל', 'חלק', 'זוג', 'הטופס', 'שישי', 'להמציא